In [28]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv
import time

odds_URL = "https://www.betfair.com/sport/football/scottish-premiership/105"
fixtures_URL = "http://api.clubelo.com/Fixtures"


def fractional_to_decimal(fractional):
    fractional = fractional.strip("\n")
    
    if fractional == "EVS":
        return np.NaN
    
    first, second = [float(x) for x in fractional.split("/")]
    
    return round(first/second+1.0, 2)

def current_odds():

    response = requests.get(odds_URL)
    soup = BeautifulSoup(response.text, 'html.parser')

    events = soup.findAll('div', attrs={"class":"event-information"})
    matches = []

    for event in events:
        teams = event.findAll("span", attrs={"class":"team-name"})
        home_name = teams[0].attrs['title']
        away_name = teams[1].attrs['title']
        home_name = home_name.replace("Utd", "United")
        away_name = away_name.replace("Utd", "United")
        home_name = home_name.replace("Co", "County")
        away_name = away_name.replace("Co", "County")
        
        odds = event.findAll("span", attrs={"class":"ui-runner-price"})

        
        over = fractional_to_decimal(odds[0].text)
        under = fractional_to_decimal(odds[1].text)
        home_odds = fractional_to_decimal(odds[2].text)
        draw_odds = fractional_to_decimal(odds[3].text)
        away_odds = fractional_to_decimal(odds[4].text)

        matches.append((home_name, away_name, home_odds, draw_odds, away_odds, over, under))

    return pd.DataFrame(matches, columns=('Home', "Away", "O(W)", "O(D)", "O(L)", "O(over2.5)", "O(under2.5)"))

def current_prob():
    
    # def current_prob():
    away = ["GD<-5","GD=-5","GD=-4","GD=-3","GD=-2","GD=-1"]
    draw = ["GD=0"]
    home = ["GD=1","GD=2","GD=3","GD=4","GD=5","GD>5"]
    over = ["GD<-5","GD=-5","GD=-4","GD=-3","GD=3","GD=4","GD=5","GD>5"]
    under = ["GD=-2","GD=-1","GD=0","GD=1","GD=2"]

    response = requests.get(fixtures_URL)
    decoded_content = response.content.decode('utf-8')
    fixtures = pd.DataFrame(csv.reader(decoded_content.splitlines(), delimiter=','))
    fixtures.columns = fixtures.iloc[0]
    fixtures = fixtures.drop(0)

    fixtures = fixtures[fixtures['Country'] == 'SCO']

    fixtures["P(W)"] = round(fixtures[home].astype(float).sum(axis=1), 2)
    fixtures["P(D)"] = round(fixtures[draw].astype(float).sum(axis=1), 2)
    fixtures["P(L)"] = round(fixtures[away].astype(float).sum(axis=1), 2)
    fixtures["P(over2.5)"] = round(fixtures[over].astype(float).sum(axis=1), 2)
    fixtures["P(under2.5)"] = round(fixtures[under].astype(float).sum(axis=1), 2)

    fixtures = fixtures[["Date","Home", "Away", "P(W)", "P(D)", "P(L)", "P(over2.5)", "P(under2.5)"]]
    
    return fixtures
    
def color_negative(v, color, limit):
        return f"color: {color};" if v < limit else "color: green;font-weight: bold"
    
def display_summary():
    summary_df = summary()
    
    styler = summary_df.style.applymap(color_negative, color='red', limit=0, subset=['Payoff(W)', 'Payoff(D)', 'Payoff(L)', 'Payoff(under2.5)', "Payoff(over2.5)"])
    styler = styler.applymap(color_negative, color='red', limit=0.03, subset=['Edge(W)', 'Edge(D)', 'Edge(L)','Edge(over2.5)', "Edge(under2.5)"])
    styler.format({
        'P(W)': '{:,.2%}'.format,
        'P(D)': '{:,.2%}'.format,
        'P(L)': '{:,.2%}'.format,
        'Edge(W)': '{:,.2%}'.format,
        'Edge(D)': '{:,.2%}'.format,
        'Edge(L)': '{:,.2%}'.format,
    })
    
    return styler

def summary(bet_amount=1):
    

    prob = current_prob()
    odds = current_odds()
    
    summary_df = odds.merge(prob)
    
    summary_df["Payoff(W)"] = (summary_df["P(W)"]*(summary_df["O(W)"]-bet_amount))-((1-summary_df["P(W)"])*bet_amount)
    summary_df["Payoff(D)"] = (summary_df["P(D)"]*(summary_df["O(D)"]-bet_amount))-((1-summary_df["P(D)"])*bet_amount)
    summary_df["Payoff(L)"] = (summary_df["P(L)"]*(summary_df["O(L)"]-bet_amount))-((1-summary_df["P(L)"])*bet_amount)
    summary_df["Payoff(under2.5)"] = (summary_df["P(under2.5)"]*(summary_df["O(under2.5)"]-bet_amount))-((1-summary_df["P(under2.5)"])*bet_amount)  
    summary_df["Payoff(over2.5)"] = (summary_df["P(over2.5)"]*(summary_df["O(over2.5)"]-bet_amount))-((1-summary_df["P(over2.5)"])*bet_amount)  
    
    summary_df["Edge(W)"] = summary_df["P(W)"] - (1/summary_df["O(W)"])
    summary_df["Edge(D)"] = summary_df["P(D)"] - (1/summary_df["O(D)"])
    summary_df["Edge(L)"] = summary_df["P(L)"] - (1/summary_df["O(L)"])
    summary_df["Edge(under2.5)"] = summary_df["P(under2.5)"] - (1/summary_df["O(under2.5)"])
    summary_df["Edge(over2.5)"] = summary_df["P(over2.5)"] - (1/summary_df["O(over2.5)"])
    
    
    #[]
    return summary_df.head(len(summary_df)).iloc[:,[7,0,1,13,18,14,19,15,20,16,21,17,22]]


display_summary()

,Date,Home,Away,Payoff(W),Edge(W),Payoff(D),Edge(D),Payoff(L),Edge(L),Payoff(under2.5),Edge(under2.5),Payoff(over2.5),Edge(over2.5)
0,2023-02-04,Livingston,Kilmarnock,0.21,9.55%,-0.194,-6.26%,-0.354,-10.41%,0.386,0.233939,-0.648,-0.294545
1,2023-02-04,St Mirren,Hibernian,0.176,6.29%,-0.0575,-1.77%,-0.275,-11.00%,0.5397,0.311965,-0.769,-0.36619
2,2023-02-04,Hearts,Dundee United,-0.076,-4.61%,-0.05,-1.32%,-0.05,-1.00%,0.7015,0.342195,-0.7025,-0.401429
3,2023-02-04,Aberdeen,Motherwell,-0.103,-5.28%,-0.082,-2.41%,-0.025,-0.67%,0.6269,0.335241,-0.753,-0.396316
4,2023-02-04,Rangers,Ross County,-0.0343,-3.09%,-0.145,-1.53%,-0.52,-3.25%,0.6128,0.212778,-0.384,-0.274286
5,2023-02-05,St Johnstone,Celtic,0.04,0.31%,-0.025,-0.38%,-0.076,-6.33%,0.9872,0.342778,-0.566,-0.404286


In [27]:
current_prob()

,Date,Home,Away,P(W),P(D),P(L),P(over2.5),P(under2.5)
109,2023-02-04,Hearts,Dundee United,0.56,0.25,0.19,0.17,0.83
110,2023-02-04,Rangers,Ross County,0.87,0.09,0.03,0.44,0.56
111,2023-02-04,St Mirren,Hibernian,0.42,0.29,0.29,0.11,0.89
112,2023-02-04,Aberdeen,Motherwell,0.46,0.27,0.26,0.13,0.87
113,2023-02-04,Livingston,Kilmarnock,0.55,0.26,0.19,0.16,0.84
169,2023-02-05,St Johnstone,Celtic,0.08,0.15,0.77,0.31,0.69


In [29]:
current_odds()

,Home,Away,O(W),O(D),O(L),O(over2.5),O(under2.5)
0,Livingston,Kilmarnock,2.20,3.10,3.40,2.20,1.65
1,St Mirren,Hibernian,2.80,3.25,2.50,2.10,1.73
2,Hearts,Dundee United,1.65,3.80,5.00,1.75,2.05
3,Aberdeen,Motherwell,1.95,3.40,3.75,1.90,1.87
4,Rangers,Ross County,1.11,9.50,16.00,1.40,2.88
5,St Johnstone,Celtic,13.00,6.50,1.20,1.40,2.88
